<img src="https://github.com/pmservice/ai-openscale-tutorials/raw/master/notebooks/images/banner.png" align="left" alt="banner">

# One API call to configure SageMaker asset subscription

This notebook shows how to log the payload for the model deployed on custom model serving engine using AI OpenScale python sdk.

Contents
- [1. Setup](#setup)
- [2. Binding machine learning engine](#binding)
- [3. Subscriptions](#subscription)
- [4. Scoring and payload logging](#scoring)
- [5. Feedback logging](#feedback)
- [6. Data Mart](#datamart)

<a id="setup"></a>
## 1. Setup

Import and initiate.

In [7]:
!pip install --upgrade ibm-ai-openscale --no-cache | tail -n 1

You are using pip version 9.0.1, however version 19.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
Requirement already up-to-date: docutils>=0.10 in /Applications/anaconda/envs/python35/lib/python3.5/site-packages (from ibm-cos-sdk-core==2.*,>=2.0.0->ibm-cos-sdk->watson-machine-learning-client->ibm-ai-openscale)


In [8]:
import json
from ibm_ai_openscale import APIClient
from ibm_ai_openscale.supporting_classes import PayloadRecord
from ibm_ai_openscale.engines import *
from ibm_ai_openscale.utils import *

#### Let's define some constants required to set up data mart:

- AIOS_CREDENTIALS
- POSTGRES_CREDENTIALS
- SCHEMA_NAME

In [2]:
AIOS_CREDENTIALS = {
  "url": "https://api.aiopenscale.cloud.ibm.com",
  "instance_guid": "***",
  "apikey": "***"
}

In [9]:
# The code was removed by Watson Studio for sharing.

In [1]:
POSTGRES_CREDENTIALS = {
    "db_type": "postgresql",
    "uri_cli_1": "xxx",
    "maps": [],
    "instance_administration_api": {
        "instance_id": "xxx",
        "root": "xxx",
        "deployment_id": "xxx"
    },
    "name": "xxx",
    "uri_cli": "xxx",
    "uri_direct_1": "xxx",
    "ca_certificate_base64": "xxx",
    "deployment_id": "xxx",
    "uri": "xxx"
}

In [10]:
# The code was removed by Watson Studio for sharing.

In [11]:
SCHEMA_NAME = 'data_mart_for_one_call'

Create schema for data mart.

In [ ]:
create_postgres_schema(postgres_credentials=POSTGRES_CREDENTIALS, schema_name=SCHEMA_NAME)

In [13]:
client = APIClient(AIOS_CREDENTIALS)

In [14]:
client.version

'2.0.18'

### 1.2 DataMart setup

In [15]:
client.data_mart.setup(db_credentials=POSTGRES_CREDENTIALS, schema=SCHEMA_NAME)

Warning during setup of data mart.
Status code: 409, body: {"trace":"ZmYzMGNkYzktOWNlNi00YzNkLTk3MDQtYTRhYjBhN2E4NzI4","errors":[{"code":"AISCS0004W","message":"Data Mart with this id is already defined"}]}


<a id="binding"></a>
## 2. Bind machine learning engines

### 2.1 Bind  `SageMaker` machine learning engine

Provide credentials using following fields:
- `access_key_id`
- `secret_access_key`
- `region`

In [3]:
SAGEMAKER_ENGINE_CREDENTIALS = {
        "access_key_id": "***",
        "secret_access_key": "***",
        "region": "***"
}

In [16]:
# The code was removed by Watson Studio for sharing.

In [17]:
binding_uid = client.data_mart.bindings.add('My SageMaker engine', SageMakerMachineLearningInstance(SAGEMAKER_ENGINE_CREDENTIALS))

In [18]:
client.data_mart.bindings.list()

d47e97b7-07d7-4874-b604-e47ff1a0cd26,My SageMaker engine,amazon_sagemaker,2019-02-14T12:50:02.856Z
a73bf76d-a663-448a-b771-4f651f73d54e,WML instance,watson_machine_learning,2019-02-13T10:34:55.751Z
91d749e3-8551-45da-84fc-385e7ed2ba26,My Azure ML Studio engine,azure_machine_learning,2019-02-07T12:44:20.125Z


<a id="subsciption"></a>
## 3. Subscriptions

### 3.1 Load the configuration file content as dict

In [4]:
!wget https://raw.githubusercontent.com/pmservice/ai-openscale-tutorials/master/assets/sagemaker_native_multiclass_breast-cancer_all_monitors_sub_configuration.json

--2019-02-14 14:05:50--  https://raw.githubusercontent.com/pmservice/ai-openscale-tutorials/master/assets/sagemaker_native_multiclass_breast-cancer_all_monitors_sub_configuration.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.48.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.48.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 33498 (33K) [text/plain]
Saving to: ‘sagemaker_native_multiclass_breast-cancer_all_monitors_sub_configuration.json’

100%[======================================>] 33,498      --.-K/s   in 0.002s  

2019-02-14 14:05:50 (14.5 MB/s) - ‘sagemaker_native_multiclass_breast-cancer_all_monitors_sub_configuration.json’ saved [33498/33498]



In [23]:
configuration_file_path = 'sagemaker_native_multiclass_breast-cancer_all_monitors_sub_configuration.json'

with open(configuration_file_path, 'r') as fp:
    subscription_configuration = json.load(fp)

### Preview the configuration file content

In [24]:
subscription_configuration

{'asset': {'asset_id': '0530ab0cd4f4dd5486b19c08df8b6914',
  'asset_type': 'model',
  'created_at': '2018-10-10T14:31:44.348Z',
  'name': 'DEMO-multi-classification-2018-10-10-14-26-26',
  'url': 's3://sagemaker-us-east-1-014862798213/sagemaker/DEMO-breast-cancer-prediction/DEMO-multi-classification-2018-10-10-14-26-26/output/model.tar.gz'},
 'asset_properties': {'categorical_fields': [],
  'feature_fields': ['radius_mean',
   'texture_mean',
   'perimeter_mean',
   'area_mean',
   'smoothness_mean',
   'compactness_mean',
   'concavity_mean',
   'concave points_mean',
   'symmetry_mean',
   'fractal_dimension_mean',
   'radius_se',
   'texture_se',
   'perimeter_se',
   'area_se',
   'smoothness_se',
   'compactness_se',
   'concavity_se',
   'concave points_se',
   'symmetry_se',
   'fractal_dimension_se',
   'radius_worst',
   'texture_worst',
   'perimeter_worst',
   'area_worst',
   'smoothness_worst',
   'compactness_worst',
   'concavity_worst',
   'concave points_worst',
   'sy

### 3.2 Add subscriptions from configuration file

In [21]:
subscription = client.data_mart.subscriptions.import_configuration(binding_uid=binding_uid, configuration_data=subscription_configuration)

#### List subscriptions

In [25]:
client.data_mart.subscriptions.list()

0530ab0cd4f4dd5486b19c08df8b6914,DEMO-multi-classification-2018-10-10-14-26-26,model,d47e97b7-07d7-4874-b604-e47ff1a0cd26,2019-02-14T12:50:55.426Z
0f1d511c-b121-44d1-b3f7-c3ad258211f5,AIOS Spark German Risk Model - Final,model,a73bf76d-a663-448a-b771-4f651f73d54e,2019-02-13T10:36:38.050Z
085460ef94636166aea5800e9ea26168,GermanCreditRisk.2019.1.9.10.41.58.611,model,91d749e3-8551-45da-84fc-385e7ed2ba26,2019-02-07T12:46:42.998Z


### 3.3 Export subscription configuration

In [26]:
exported_configuration = client.data_mart.subscriptions.export_configuration(binding_uid=binding_uid, subscription_uid=subscription.uid)

In [27]:
exported_configuration

{'asset': {'asset_id': '0530ab0cd4f4dd5486b19c08df8b6914',
  'asset_type': 'model',
  'created_at': '2018-10-10T14:31:44.348Z',
  'name': 'DEMO-multi-classification-2018-10-10-14-26-26',
  'url': 's3://sagemaker-us-east-1-014862798213/sagemaker/DEMO-breast-cancer-prediction/DEMO-multi-classification-2018-10-10-14-26-26/output/model.tar.gz'},
 'asset_properties': {'categorical_fields': [],
  'feature_fields': ['radius_mean',
   'texture_mean',
   'perimeter_mean',
   'area_mean',
   'smoothness_mean',
   'compactness_mean',
   'concavity_mean',
   'concave points_mean',
   'symmetry_mean',
   'fractal_dimension_mean',
   'radius_se',
   'texture_se',
   'perimeter_se',
   'area_se',
   'smoothness_se',
   'compactness_se',
   'concavity_se',
   'concave points_se',
   'symmetry_se',
   'fractal_dimension_se',
   'radius_worst',
   'texture_worst',
   'perimeter_worst',
   'area_worst',
   'smoothness_worst',
   'compactness_worst',
   'concavity_worst',
   'concave points_worst',
   'sy

<a id="scoring"></a>
## 4. Scoring and payload logging

### 4.1 Score the cancer model and measure response time

In [28]:
import requests
import time
import json
import boto3

binding_details = client.data_mart.bindings.get_details(binding_uid=binding_uid)
subscription_details = subscription.get_details()

access_id = binding_details['entity']['credentials']['access_key_id']
access_key = binding_details['entity']['credentials']['secret_access_key']
region = binding_details['entity']['credentials']['region']
endpoint_name = subscription_details['entity']['deployments'][0]['name']
runtime = boto3.client('sagemaker-runtime', region_name=region, aws_access_key_id=access_id, aws_secret_access_key=access_key)

fields = ['radius_mean', 'texture_mean', 'perimeter_mean', 'area_mean', 'smoothness_mean', 'compactness_mean',
          'concavity_mean', 'concave points_mean', 'symmetry_mean', 'fractal_dimension_mean', 'radius_se',
          'texture_se', 'perimeter_se', 'area_se', 'smoothness_se', 'compactness_se', 'concavity_se',
          'concave points_se', 'symmetry_se', 'fractal_dimension_se', 'radius_worst', 'texture_worst',
          'perimeter_worst', 'area_worst', 'smoothness_worst', 'compactness_worst', 'concavity_worst',
          'concave points_worst', 'symmetry_worst', 'fractal_dimension_worst']
        
payload = "17.02,23.98,112.8,899.3,0.1197,0.1496,0.2417,0.1203,0.2248,0.06382,0.6009,1.398,3.999,67.78,0.008268,0.03082,0.05042,0.01112,0.02102,0.003854,20.88,32.09,136.1,1344,0.1634,0.3559,0.5588,0.1847,0.353,0.08482\n17.02,23.98,112.8,899.3,0.1197,0.1496,0.2417,0.1203,0.2248,0.06382,0.6009,1.398,3.999,67.78,0.008268,0.03082,0.05042,0.01112,0.02102,0.003854,20.88,32.09,136.1,1344,0.1634,0.3559,0.5588,0.1847,0.353,0.08482"

start_time = time.time()
response = runtime.invoke_endpoint(EndpointName=endpoint_name, ContentType='text/csv', Body=payload)
response_time = int((time.time() - start_time)*1000)
result = json.loads(response['Body'].read().decode())

print(json.dumps(result, indent=2))

{
  "predictions": [
    {
      "score": [
        5.532644564709699e-09,
        1.0
      ],
      "predicted_label": 1.0
    },
    {
      "score": [
        5.532644564709699e-09,
        1.0
      ],
      "predicted_label": 1.0
    }
  ]
}


### 4.2 Store the request and response in payload logging table

#### Transform the model's input and output to the format compatible with AI OpenScale standard.

In [29]:
values = []

for v in payload.split('\n'):
    values.append([float(s) for s in v.split(',')])

request_data = {'fields': fields, 'values': values}

response_data = {'fields': list(result['predictions'][0]),
            'values': [list(x.values()) for x in result['predictions']]}

#### Store the payload using Python SDK

**Hint:** You can embed payload logging code into your custom deployment so it is logged automatically each time you score the model.

In [30]:
records_list = [PayloadRecord(request=request_data, response=response_data, response_time=response_time), 
                PayloadRecord(request=request_data, response=response_data, response_time=response_time)]

for i in range(1, 10):
    records_list.append(PayloadRecord(request=request_data, response=response_data, response_time=response_time))

subscription.payload_logging.store(records=records_list)

<a id="feedback"></a>
## 5. Feedback logging & quality (accuracy) monitoring

### Feedback records logging

The feedback records can be send to feedback table using below code.

In [32]:
feedback_records = []

fields = ['radius_mean', 'texture_mean', 'perimeter_mean', 'area_mean', 'smoothness_mean', 'compactness_mean',
          'concavity_mean', 'concave points_mean', 'symmetry_mean', 'fractal_dimension_mean', 'radius_se',
          'texture_se', 'perimeter_se', 'area_se', 'smoothness_se', 'compactness_se', 'concavity_se',
          'concave points_se', 'symmetry_se', 'fractal_dimension_se', 'radius_worst', 'texture_worst',
          'perimeter_worst', 'area_worst', 'smoothness_worst', 'compactness_worst', 'concavity_worst',
          'concave points_worst', 'symmetry_worst', 'fractal_dimension_worst', 'diagnosis']

for i in range(1, 10):
    feedback_records.append([17.02,23.98,112.8,899.3,0.1197,0.1496,0.2417,0.1203,0.2248,0.06382,0.6009,1.398,3.999,67.78,0.008268,0.03082,0.05042,0.01112,0.02102,0.003854,20.88,32.09,136.1,1344,0.1634,0.3559,0.5588,0.1847,0.353,0.08482, 1])

subscription.feedback_logging.store(feedback_data=feedback_records, fields=fields)

### Run quality monitoring on demand

By default, quality monitoring is run on hourly schedule. You can also trigger it on demand using below code.

In [33]:
run_details = subscription.quality_monitoring.run(background_mode=False)




 Waiting for end of quality monitoring run d5c9f513-a6f8-4fec-9889-08b8ce9596f2 




initializing
completed

---------------------------
 Successfully finished run 
---------------------------




### Show the quality metrics

In [34]:
subscription.quality_monitoring.show_table()

2019-02-14 12:58:10.616000+00:00,1.0,0.8,d47e97b7-07d7-4874-b604-e47ff1a0cd26,0530ab0cd4f4dd5486b19c08df8b6914,37a83f399e6dc3b9d08d7d01fe690665,Accuracy_evaluation_d5c9f513-a6f8-4fec-9889-08b8ce9596f2,


Get all calculated metrics.

In [35]:
deployment_uid = subscription.get_deployment_uids()[0]

In [36]:
subscription.quality_monitoring.get_metrics(deployment_uid=deployment_uid)

{'end': '2019-02-14T12:58:26.874423Z',
 'metrics': [{'process': 'Accuracy_evaluation_d5c9f513-a6f8-4fec-9889-08b8ce9596f2',
   'timestamp': '2019-02-14T12:58:10.616Z',
   'value': {'metrics': [{'name': 'weightedTruePositiveRate', 'value': 1.0},
     {'name': 'accuracy', 'value': 1.0},
     {'name': 'weightedFMeasure', 'value': 1.0},
     {'name': 'weightedRecall', 'value': 1.0},
     {'name': 'weightedFalsePositiveRate', 'value': None},
     {'name': 'weightedPrecision', 'value': 1.0}],
    'quality': 1.0,
    'threshold': 0.8}}],
 'start': '2019-02-14T11:50:55.426Z'}

<a id="datamart"></a>
## 6. Explainability

In [37]:
subscription.payload_logging.show_table()

33a30b30-6526-4e58-9dae-72eab044a022-1,2019-02-14 12:57:48.299540+00:00,37a83f399e6dc3b9d08d7d01fe690665,None,17.02,23.98,112.8,899.3,0.1197,0.1496,0.2417,0.1203,0.2248,0.06382,0.6009,1.398,3.999,67.78,0.008268,0.03082,0.05042,0.01112,0.02102,0.003854,20.88,32.09,136.1,1344.0,0.1634,0.3559,0.5588,0.1847,0.353,0.08482,"[5.532644564709699e-09, 1.0]",1.0
33a30b30-6526-4e58-9dae-72eab044a022-2,2019-02-14 12:57:48.299540+00:00,37a83f399e6dc3b9d08d7d01fe690665,None,17.02,23.98,112.8,899.3,0.1197,0.1496,0.2417,0.1203,0.2248,0.06382,0.6009,1.398,3.999,67.78,0.008268,0.03082,0.05042,0.01112,0.02102,0.003854,20.88,32.09,136.1,1344.0,0.1634,0.3559,0.5588,0.1847,0.353,0.08482,"[5.532644564709699e-09, 1.0]",1.0
c037d27e-b442-4f86-827e-d1a5b346700f-1,2019-02-14 12:57:48.299505+00:00,37a83f399e6dc3b9d08d7d01fe690665,None,17.02,23.98,112.8,899.3,0.1197,0.1496,0.2417,0.1203,0.2248,0.06382,0.6009,1.398,3.999,67.78,0.008268,0.03082,0.05042,0.01112,0.02102,0.003854,20.88,32.09,136.1,1344.0,0.1634,0.3559,0.5588,0.1847,0.353,0.08482,"[5.532644564709699e-09, 1.0]",1.0
c037d27e-b442-4f86-827e-d1a5b346700f-2,2019-02-14 12:57:48.299505+00:00,37a83f399e6dc3b9d08d7d01fe690665,None,17.02,23.98,112.8,899.3,0.1197,0.1496,0.2417,0.1203,0.2248,0.06382,0.6009,1.398,3.999,67.78,0.008268,0.03082,0.05042,0.01112,0.02102,0.003854,20.88,32.09,136.1,1344.0,0.1634,0.3559,0.5588,0.1847,0.353,0.08482,"[5.532644564709699e-09, 1.0]",1.0
b372ba7b-a164-4bef-b6cc-998eb2153f19-1,2019-02-14 12:57:48.299472+00:00,37a83f399e6dc3b9d08d7d01fe690665,None,17.02,23.98,112.8,899.3,0.1197,0.1496,0.2417,0.1203,0.2248,0.06382,0.6009,1.398,3.999,67.78,0.008268,0.03082,0.05042,0.01112,0.02102,0.003854,20.88,32.09,136.1,1344.0,0.1634,0.3559,0.5588,0.1847,0.353,0.08482,"[5.532644564709699e-09, 1.0]",1.0
b372ba7b-a164-4bef-b6cc-998eb2153f19-2,2019-02-14 12:57:48.299472+00:00,37a83f399e6dc3b9d08d7d01fe690665,None,17.02,23.98,112.8,899.3,0.1197,0.1496,0.2417,0.1203,0.2248,0.06382,0.6009,1.398,3.999,67.78,0.008268,0.03082,0.05042,0.01112,0.02102,0.003854,20.88,32.09,136.1,1344.0,0.1634,0.3559,0.5588,0.1847,0.353,0.08482,"[5.532644564709699e-09, 1.0]",1.0
a5ec0c76-a3d1-40f9-9dea-131fe0d8d98c-1,2019-02-14 12:57:48.299441+00:00,37a83f399e6dc3b9d08d7d01fe690665,None,17.02,23.98,112.8,899.3,0.1197,0.1496,0.2417,0.1203,0.2248,0.06382,0.6009,1.398,3.999,67.78,0.008268,0.03082,0.05042,0.01112,0.02102,0.003854,20.88,32.09,136.1,1344.0,0.1634,0.3559,0.5588,0.1847,0.353,0.08482,"[5.532644564709699e-09, 1.0]",1.0
a5ec0c76-a3d1-40f9-9dea-131fe0d8d98c-2,2019-02-14 12:57:48.299441+00:00,37a83f399e6dc3b9d08d7d01fe690665,None,17.02,23.98,112.8,899.3,0.1197,0.1496,0.2417,0.1203,0.2248,0.06382,0.6009,1.398,3.999,67.78,0.008268,0.03082,0.05042,0.01112,0.02102,0.003854,20.88,32.09,136.1,1344.0,0.1634,0.3559,0.5588,0.1847,0.353,0.08482,"[5.532644564709699e-09, 1.0]",1.0
8fcdf8e0-7309-440b-8bb4-aa5f34b83930-1,2019-02-14 12:57:48.299407+00:00,37a83f399e6dc3b9d08d7d01fe690665,None,17.02,23.98,112.8,899.3,0.1197,0.1496,0.2417,0.1203,0.2248,0.06382,0.6009,1.398,3.999,67.78,0.008268,0.03082,0.05042,0.01112,0.02102,0.003854,20.88,32.09,136.1,1344.0,0.1634,0.3559,0.5588,0.1847,0.353,0.08482,"[5.532644564709699e-09, 1.0]",1.0
8fcdf8e0-7309-440b-8bb4-aa5f34b83930-2,2019-02-14 12:57:48.299407+00:00,37a83f399e6dc3b9d08d7d01fe690665,None,17.02,23.98,112.8,899.3,0.1197,0.1496,0.2417,0.1203,0.2248,0.06382,0.6009,1.398,3.999,67.78,0.008268,0.03082,0.05042,0.01112,0.02102,0.003854,20.88,32.09,136.1,1344.0,0.1634,0.3559,0.5588,0.1847,0.353,0.08482,"[5.532644564709699e-09, 1.0]",1.0


### Run explainability

In [46]:
sample_transaction_id = subscription.payload_logging.get_table_content(limit=1)['scoring_id'][0]

run_details = subscription.explainability.run(transaction_id=sample_transaction_id, background_mode=False)




 Looking for explanation for 33a30b30-6526-4e58-9dae-72eab044a022-1 




in_progress......
finished

---------------------------
 Successfully finished run 
---------------------------




In [55]:
import json

print(json.dumps(run_details['entity']['contrastive_explanation'], indent=2))

{
  "is_pn_closest_point_to_pp": true,
  "pertinent_negative_features": [
    {
      "feature_name": "concavity_worst",
      "importance": "0.02350630478235246",
      "feature_value": "0.23160"
    },
    {
      "feature_name": "radius_mean",
      "importance": "0.0",
      "feature_value": "13.37"
    },
    {
      "feature_name": "texture_mean",
      "importance": "0.0",
      "feature_value": "18.84"
    },
    {
      "feature_name": "perimeter_mean",
      "importance": "0.0",
      "feature_value": "86.24"
    },
    {
      "feature_name": "area_mean",
      "importance": "0.0",
      "feature_value": "551.1"
    },
    {
      "feature_name": "smoothness_mean",
      "importance": "0.0",
      "feature_value": "0.09587"
    },
    {
      "feature_name": "compactness_mean",
      "importance": "0.0",
      "feature_value": "0.09263"
    },
    {
      "feature_name": "concavity_mean",
      "importance": "0.0",
      "feature_value": "0.06154"
    },
    {
      "feature

---

### Authors
Lukasz Cmielowski, PhD, is an Automation Architect and Data Scientist at IBM with a track record of developing enterprise-level applications that substantially increases clients' ability to turn data into actionable knowledge.